In [ ]:
"""
====================================
Create a lonboard map of NEXRAD reflectivity
====================================

An example which follows the "Create a plot of NEXRAD reflectivity", but plots with the
lonboard deck.gl python interface.

"""

print(__doc__)

# Author: Sam Gardner (samuel.gardner@ttu.edu)
# License: BSD 3 clause

from arro3.core import Array, Field, fixed_size_list_array, list_array, Table, Schema
import cmweather
from lonboard import Map, SolidPolygonLayer
from lonboard.colormap import apply_continuous_cmap
import numpy as np
import pyart
from pyart.testing import get_test_data


# download and open test data file
filename = get_test_data("Level2_KATX_20130717_1950.ar2v")
radar = pyart.io.read_nexrad_archive(filename)
# extract the first sweep of data
first_sweep = radar.extract_sweeps([0])

In [2]:
# extract the range, azimuth, and elevation of gates in the first sweep
range_center = first_sweep.range["data"]
az_center = first_sweep.azimuth["data"]
elevation = first_sweep.elevation["data"]
# Convert the gate center coordinates to latitude and longitude edges
x_edge, y_edge, _ = pyart.core.antenna_vectors_to_cartesian(
    range_center, az_center, elevation, edges=True
)
radar_lat = radar.latitude["data"][0]
radar_lon = radar.longitude["data"][0]
projparams = {"proj": "pyart_aeqd", "lon_0": radar_lon, "lat_0": radar_lat}
lons, lats = pyart.core.cartesian_to_geographic(x_edge, y_edge, projparams)

In [3]:
# Extract reflectivity data from the first sweep and fill masked values with NaN
reflectivity = first_sweep.fields["reflectivity"]["data"].filled(np.nan)
# lonboard colormaps data over a range of 0-1, so we normalize the reflectivity data by our vmin and vmax
# The ChaseSpectral colormap aligns lightness breaks with relevant meteorological phenomena
# when used with a vmin of -10 dBZ and a vmax of 80 dBZ
chasespectral_vmin = -10
chasespectral_vmax = 80
# Scale the reflectivity data to the range 0-1
reflectivity_normalized = np.ravel(
    (reflectivity - chasespectral_vmin) / (chasespectral_vmax - chasespectral_vmin)
)
# Plotting takes much less time if data below the vmin and NaN values are not plotted
reflectivity_normalized[reflectivity_normalized < 0] = np.nan
points_to_keep = ~np.isnan(reflectivity_normalized)
reflectivity_normalized = reflectivity_normalized[points_to_keep]

In [4]:
# This function gets the corners of each radar gate and returns them as a geoarrow table
# Geoarrow tables are a way to represent geospatial data in Apache Arrow, which is used by lonboard
# for rendering geospatial data. This function originally written by Kyle Barron
# See https://github.com/developmentseed/lonboard/discussions/643
def lon_lat_to_arrow(lon_edge, lat_edge, data_mask):
    # bottom-left: (i, j)
    bl_lon = lon_edge[:-1, :-1]
    bl_lat = lat_edge[:-1, :-1]

    # bottom-right: (i+1, j)
    br_lon = lon_edge[1:, :-1]
    br_lat = lat_edge[1:, :-1]

    # top-right: (i+1, j+1)
    tr_lon = lon_edge[1:, 1:]
    tr_lat = lat_edge[1:, 1:]

    # top-left: (i, j+1)
    tl_lon = lon_edge[:-1, 1:]
    tl_lat = lat_edge[:-1, 1:]

    lons_to_poly = np.array(
        [bl_lon.flatten(), br_lon.flatten(), tr_lon.flatten(), tl_lon.flatten()]
    )
    lats_to_poly = np.array(
        [bl_lat.flatten(), br_lat.flatten(), tr_lat.flatten(), tl_lat.flatten()]
    )
    poly_coords = np.stack([lons_to_poly, lats_to_poly], axis=-1).transpose(1, 0, 2)
    poly_coords = poly_coords[data_mask, :, :]

    poly_coords_closed = np.concat((poly_coords, poly_coords[:, 0:1, :]), axis=1)

    poly_coords_arrow = fixed_size_list_array(
        Array.from_numpy(np.ravel(poly_coords_closed)), 2
    )
    ring_offsets = Array.from_numpy(
        np.arange(0, (poly_coords_closed.shape[0] + 1) * 5, 5, dtype=np.int32)
    )
    arrow_rings = list_array(ring_offsets, poly_coords_arrow)
    geom_offsets = Array.from_numpy(
        np.arange(poly_coords_closed.shape[0] + 1, dtype=np.int32)
    )
    arrow_geoms = list_array(geom_offsets, arrow_rings)
    extension_metadata = {"ARROW:extension:name": "geoarrow.polygon"}
    field = Field(
        "geometry", arrow_geoms.type, nullable=True, metadata=extension_metadata
    )
    schema = Schema([field])
    table = Table.from_arrays([arrow_geoms], schema=schema)
    return table

In [ ]:
# Get the geoarrow table from the lon and lat data
arrow_table = lon_lat_to_arrow(lons, lats, points_to_keep)
# Create a solid polygon layer with the geoarrow table and the reflectivity data
layer = SolidPolygonLayer(
    table=arrow_table,
    get_fill_color=apply_continuous_cmap(
        reflectivity_normalized, cmweather.cm_colorblind.ChaseSpectral
    ),
)
# Plot the data
m = Map(layers=[layer])
m